In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from pandas_datareader import data as pdr
from scipy.stats import norm, t
import matplotlib.pyplot as plt

# Import data

### Getting the stock data from yahoo and compute some parameters

In [26]:
def getData(stocks, start, end):
    stockData = pdr.get_data_yahoo(stocks, start=start, end=end)
    stockData = stockData['Close']
    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return returns, meanReturns, covMatrix

# Portfolio Performance
#### We compute here the Expected Portfolio Return and the standard deviation of the portfolio

In [4]:
def portfolioPerformance(weights, meanReturns, covMatrix, Time):
    returns = np.sum(meanReturns*weights)*Time
    std = np.sqrt( np.dot(weights.T, np.dot(covMatrix, weights)) ) * np.sqrt(Time)
    return returns, std

### Compute returns data

In [5]:
stockList = ['CBA', 'BHP', 'TLS', 'NAB', 'WBC', 'STO']
stocks = [stock+'.AX' for stock in stockList]
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=800)
returns, meanReturns, covMatrix = getData(stocks, start=startDate, end=endDate)
returns = returns.dropna()

In [27]:
returns

Symbols,CBA.AX,BHP.AX,TLS.AX,NAB.AX,WBC.AX,STO.AX,portfolio
Date,,,,,,,
2020-07-21,0.033894,0.010943,0.000000,0.025267,0.024887,0.034884,0.025329
2020-07-22,-0.004031,-0.033505,-0.002941,-0.008215,-0.006071,0.022472,-0.006759
2020-07-23,-0.005261,-0.006667,-0.011799,0.006626,0.004442,0.034799,0.002237
2020-07-24,-0.011934,-0.004564,-0.005970,-0.013714,-0.018242,-0.012389,-0.013698
2020-07-27,-0.005078,0.008630,0.009009,-0.001668,-0.002252,-0.010753,-0.001788
...,...,...,...,...,...,...,...
2022-09-16,-0.008731,-0.018069,-0.012953,0.004695,-0.000464,-0.023869,-0.004045
2022-09-19,0.004563,-0.006046,-0.005249,-0.008678,0.000929,-0.007722,-0.001790
2022-09-20,0.019755,0.036234,0.002639,0.017508,0.015777,0.015564,0.017986


### Compute random weight for each asset that will helps us compute the portfolio

In [7]:
weights = np.random.random(len(returns.columns))
weights /= np.sum(weights)
returns['portfolio'] = returns.dot(weights)

In [8]:
returns

Symbols,CBA.AX,BHP.AX,TLS.AX,NAB.AX,WBC.AX,STO.AX,portfolio
Date,,,,,,,
2020-07-21,0.033894,0.010943,0.000000,0.025267,0.024887,0.034884,0.025329
2020-07-22,-0.004031,-0.033505,-0.002941,-0.008215,-0.006071,0.022472,-0.006759
2020-07-23,-0.005261,-0.006667,-0.011799,0.006626,0.004442,0.034799,0.002237
2020-07-24,-0.011934,-0.004564,-0.005970,-0.013714,-0.018242,-0.012389,-0.013698
2020-07-27,-0.005078,0.008630,0.009009,-0.001668,-0.002252,-0.010753,-0.001788
...,...,...,...,...,...,...,...
2022-09-16,-0.008731,-0.018069,-0.012953,0.004695,-0.000464,-0.023869,-0.004045
2022-09-19,0.004563,-0.006046,-0.005249,-0.008678,0.000929,-0.007722,-0.001790
2022-09-20,0.019755,0.036234,0.002639,0.017508,0.015777,0.015564,0.017986


### Compute historical_Var on one asset or for the entire portfolio

In [33]:
def VaR(returns, alpha=5):
    """
    Read in a pandas dataframe of returns / a pandas series of returns
    Output the percentile of the distribution at the given alpha confidence level
    """
    if isinstance(returns, pd.Series):
        return np.percentile(returns, alpha)
    # A passed user-defined-function will be passed a Series for evaluation.
    elif isinstance(returns, pd.DataFrame):
        return returns.aggregate(historicalVaR, alpha=alpha)
    else:
        raise TypeError("Expected returns to be dataframe or series")

In [34]:
VaR(returns, alpha=5)

Symbols
CBA.AX      -0.034983
BHP.AX      -0.060839
TLS.AX      -0.026893
NAB.AX      -0.031172
WBC.AX      -0.034490
STO.AX      -0.050145
portfolio   -0.028051
dtype: float64

In [36]:
VaR(returns['portfolio'], alpha=5)

-0.019250781362722725

### Compute historical_CVar on one asset or for the entire portfolio

In [31]:
def CVaR(returns, alpha=5):
    """
    Read in a pandas dataframe of returns / a pandas series of returns
    Output the CVaR for dataframe / series
    """
    if isinstance(returns, pd.Series):
        belowVaR = returns <= historicalVaR(returns, alpha=alpha)
        return returns[belowVaR].mean()
    # A passed user-defined-function will be passed a Series for evaluation.
    elif isinstance(returns, pd.DataFrame):
        return returns.aggregate(historicalCVaR, alpha=alpha)
    else:
        raise TypeError("Expected returns to be dataframe or series")

In [32]:
CVaR(returns, alpha=5)

Symbols
CBA.AX      -0.029814
BHP.AX      -0.047119
TLS.AX      -0.025651
NAB.AX      -0.028399
WBC.AX      -0.030292
STO.AX      -0.043901
portfolio   -0.024904
dtype: float64

### We Apply now our var and cvar models to an itinial investement = 10000
#### What we want to know is given the level of risk alpha= 5 what is the worst case scenario and the expectation of loss in this worst case scenario

In [23]:
# 10 days
Time = 10
hVaR = -VaR(returns['portfolio'], alpha=5)*np.sqrt(Time)
hCVaR = -CVaR(returns['portfolio'], alpha=5)*np.sqrt(Time)
pRet, pStd = portfolioPerformance(weights, meanReturns, covMatrix, Time)
InitialInvestment = 10000
print('Expected Portfolio Return:      ', round(InitialInvestment*pRet,2))
print('Value at Risk 95th CI    :      ', round(InitialInvestment*hVaR,2))
print('Conditional VaR 95th CI  :      ', round(InitialInvestment*hCVaR,2))

Expected Portfolio Return:       6.04
Value at Risk 95th CI    :       192.51
Conditional VaR 95th CI  :       249.04


#### Explanation: 
##### Over a 10 days time horizon, we expect to make 6.04 and at the 95% interval confidence we are happy to not lose anymore thant 249.04